In [2]:
!pip install git+https://github.com/LUFFY-02/cuda.git

  Cloning https://github.com/LUFFY-02/cuda.git to /tmp/pip-req-build-dk691gta
  Running command git clone --filter=blob:none --quiet https://github.com/LUFFY-02/cuda.git /tmp/pip-req-build-dk691gta
  Resolved https://github.com/LUFFY-02/cuda.git to commit aa845e22f008c8f70745bd6d67f6c42c50ac5363
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=0bb6440415b243c8d81326ce6abb71783ed694c11d09dfd8f4febe36c5a67070
  Stored in directory: /tmp/pip-ephem-wheel-cache-jcebxqgn/wheels/2f/c4/fc/9c6b8e2f4a3330ee0b57405367cbbd5a1245b4bf2ca2b5b5b9
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
// checking that cuda is working
#include <iostream>
int main(){
  std::cout << "Hello World\n";
  return 0;
}

Hello World



In [ ]:
%%cu
#include <iostream>
using namespace std;

__global__ void add(int* A, int* B, int* C, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < size) {
        C[tid] = A[tid] + B[tid];
    }
}


void initialize(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        vector[i] = rand() % 10;
    }
}

void print(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        cout << vector[i] << " ";
    }
    cout << endl;
}

int main() {
    int N = 4;
    int* A, * B, * C;

    int vectorSize = N;
    size_t vectorBytes = vectorSize * sizeof(int);

    A = new int[vectorSize];
    B = new int[vectorSize];
    C = new int[vectorSize];

    initialize(A, vectorSize);
    initialize(B, vectorSize);

    cout << "Vector A: ";
    print(A, N);
    cout << "Vector B: ";
    print(B, N);

    int* X, * Y, * Z;
    cudaMalloc(&X, vectorBytes);
    cudaMalloc(&Y, vectorBytes);
    cudaMalloc(&Z, vectorBytes);

    cudaMemcpy(X, A, vectorBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(Y, B, vectorBytes, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    add<<<blocksPerGrid, threadsPerBlock>>>(X, Y, Z, N);

    cudaMemcpy(C, Z, vectorBytes, cudaMemcpyDeviceToHost);

    cout << "Addition: ";
    print(C, N);

    delete[] A;
    delete[] B;
    delete[] C;

    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}


Vector A: 3 6 7 5 
Vector B: 3 5 6 2 
Addition: 6 11 13 7 



In [6]:
%%cu
#include <iostream>
using namespace std;

__global__ void add(int* A, int* B, int* C, int size) {                           // will run on GPU
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < size) {
        C[tid] = A[tid] + B[tid];
    }
}


void initialize(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        vector[i] = rand() % 10;
    }
}

void print(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        cout << vector[i] << " ";
    }
    cout << endl;
}

int main() {
    int N = 4;
    int* A, * B, * C;

    int vectorSize = N;
    size_t vectorBytes = vectorSize * sizeof(int);                              // memory consumed by that array

    A = new int[vectorSize];
    B = new int[vectorSize];
    C = new int[vectorSize];
    initialize(A, vectorSize);
    initialize(B, vectorSize);

    cout << "Vector A: ";
    print(A, N);
    cout << "Vector B: ";
    print(B, N);

    int* X, * Y, * Z;
    cudaMalloc(&X, vectorBytes);
    cudaMalloc(&Y, vectorBytes);
    cudaMalloc(&Z, vectorBytes);

    cudaMemcpy(X, A, vectorBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(Y, B, vectorBytes, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    add<<<blocksPerGrid, threadsPerBlock>>>(X, Y, Z, N);

    cudaMemcpy(C, Z, vectorBytes, cudaMemcpyDeviceToHost);

    cout << "Addition: ";
    print(C, N);

    delete[] A;                                                                 //deallocates the memory on the host
    delete[] B;
    delete[] C;

    cudaFree(X);                                                                //deallocates the memory on device (GPU)
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}


Vector A: 3 6 7 5 
Vector B: 3 5 6 2 
Addition: 6 11 13 7 

